<a href="https://colab.research.google.com/github/SpyDplayz/Physician_Notetaker/blob/master/PhysicianNotetaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
import spacy
import yake
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

# Load NLP model for NER
nlp = spacy.load("en_core_web_sm")

def extract_medical_entities(text):
    doc = nlp(text)
    entities = {"Symptoms": [], "Diagnosis": [], "Treatment": [], "Prognosis": []}
    for ent in doc.ents:
        if ent.label_ in ["SYMPTOM", "DIAGNOSIS", "TREATMENT", "PROGNOSIS"]:
            entities[ent.label_].append(ent.text)
    return entities

# Summarization Model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_text(text):
    summary = summarizer(text, max_length=50, min_length=10, do_sample=False)
    return summary[0]['summary_text']

# Keyword Extraction
def extract_keywords(text):
    kw_extractor = yake.KeywordExtractor()
    keywords = kw_extractor.extract_keywords(text)
    return [kw[0] for kw in keywords]

# Sentiment Analysis Model
sentiment_classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

def analyze_sentiment(text):
    sentiment = sentiment_classifier(text)
    return sentiment[0]['label']

# Intent Classification
intent_labels = ["Seeking reassurance", "Reporting symptoms", "Expressing concern"]
intent_texts = [
    "I am worried about my health", "I have pain in my back", "I need help with my diagnosis"
]
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(intent_texts)
y_train = np.array([2, 1, 0])
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

def classify_intent(text):
    X_test = vectorizer.transform([text])
    pred = classifier.predict(X_test)
    return intent_labels[pred[0]]

# SOAP Note Generation
def generate_soap_note(text):
    entities = extract_medical_entities(text)
    return {
        "Subjective": {
            "Chief_Complaint": ", ".join(entities["Symptoms"]),
            "History_of_Present_Illness": summarize_text(text)
        },
        "Objective": {
            "Physical_Exam": "Normal range of motion, no tenderness.",
            "Observations": "Patient appears in normal health."
        },
        "Assessment": {
            "Diagnosis": ", ".join(entities["Diagnosis"]),
            "Severity": "Mild, improving"
        },
        "Plan": {
            "Treatment": "Physiotherapy, pain relief medications as needed.",
            "Follow-Up": "Return if symptoms worsen."
        }
    }

# Example Test
text = "Patient slipped on ice, sustaining a wrist fracture. Underwent X-ray confirmation and has been placed in a cast for four weeks."
print("\nExtracted Medical Entities:", extract_medical_entities(text))
print("\nSummarized Text:", summarize_text(text))
print("\nKeywords:", extract_keywords(text))
print("\nSentiment Analysis:", analyze_sentiment(text))
print("\nIntent Classification:", classify_intent(text))
print("\nSOAP Note:", generate_soap_note(text))


Device set to use cpu
Device set to use cpu
Your max_length is set to 50, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)



Extracted Medical Entities: {'Symptoms': [], 'Diagnosis': [], 'Treatment': [], 'Prognosis': []}

Summarized Text: Patient slipped on ice, sustaining a wrist fracture. Underwent X-ray confirmation and has been placed in a cast for four weeks.

Keywords: ['Patient slipped', 'slipped on ice', 'sustaining a wrist', 'wrist fracture', 'Underwent X-ray confirmation', 'Underwent X-ray', 'Patient', 'ice', 'sustaining', 'fracture', 'X-ray confirmation', 'slipped', 'wrist', 'X-ray', 'Underwent', 'weeks', 'confirmation', 'cast']


Your max_length is set to 50, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)



Sentiment Analysis: NEGATIVE

Intent Classification: Reporting symptoms

SOAP Note: {'Subjective': {'Chief_Complaint': '', 'History_of_Present_Illness': 'Patient slipped on ice, sustaining a wrist fracture. Underwent X-ray confirmation and has been placed in a cast for four weeks.'}, 'Objective': {'Physical_Exam': 'Normal range of motion, no tenderness.', 'Observations': 'Patient appears in normal health.'}, 'Assessment': {'Diagnosis': '', 'Severity': 'Mild, improving'}, 'Plan': {'Treatment': 'Physiotherapy, pain relief medications as needed.', 'Follow-Up': 'Return if symptoms worsen.'}}


Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


📌 **Extracted Medical Entities:** {'Symptoms': [], 'Diagnosis': [], 'Treatment': [], 'Prognosis': ['Expected recovery in four weeks']}
📌 **Summarized Text:** Patient: I had a car accident. My neck and back hurt a lot for four weeks. Did you receive treatment?
📌 **Keywords:** ['Patient', 'accident', 'neck', 'back', 'hurt', 'four', 'weeks', 'Doctor', 'receive', 'treatment', 'Patient', 'physiotherapy', 'sessions', 'only', 'have', 'occasional', 'back', 'pain']
📌 **Sentiment Analysis:** Anxious
📌 **Intent Classification:** Reporting symptoms
📌 **Structured Summary:** {'Patient_Name': 'Janet Jones', 'Symptoms': [], 'Diagnosis': 'Whiplash injury', 'Treatment': [], 'Current_Status': 'Occasional backache', 'Prognosis': 'Full recovery expected within six months'}
📌 **SOAP Note:** {'Subjective': {'Chief_Complaint': 'Neck and back pain', 'History_of_Present_Illness': '\nPatient: I had a car accident. My neck and back hurt a lot for four weeks.\nDoctor: Did you receive treatment?\nPatient: Yes, I h